In [1]:
!pip install transformers

In [3]:
import os

In [5]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [7]:
model_name= "google/gemma-1.1-2b-it"

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [9]:
tokenizer("Hello, World!")

{'input_ids': [2, 4521, 235269, 3855, 235341], 'attention_mask': [1, 1, 1, 1, 1]}

In [10]:
input_conversation = [
    {"role": "user", "content": "Which is the best place to learn GenAI?"},
    {"role":"assistant","content":"The best place to learn AI is"}
    ]

In [11]:
input_tokens = tokenizer.apply_chat_template(
    conversation=input_conversation,
    tokenize=True,
)

input_tokens

[2,
 106,
 1645,
 108,
 13033,
 603,
 573,
 1963,
 2040,
 577,
 3918,
 7472,
 11716,
 235336,
 107,
 108,
 106,
 2516,
 108,
 651,
 1963,
 2040,
 577,
 3918,
 16481,
 603,
 107,
 108]

In [12]:
input_detokens = tokenizer.apply_chat_template(
    conversation=input_conversation,
    tokenize=False,
    continue_final_message=True,
)

input_detokens

'<bos><start_of_turn>user\nWhich is the best place to learn GenAI?<end_of_turn>\n<start_of_turn>model\nThe best place to learn AI is'

In [13]:
output_lable = "GenAI Cohort .0 by ChaiCode and Anshul"
full_converstion = input_detokens + output_lable + tokenizer.eos_token
full_converstion

'<bos><start_of_turn>user\nWhich is the best place to learn GenAI?<end_of_turn>\n<start_of_turn>model\nThe best place to learn AI isGenAI Cohort .0 by ChaiCode and Anshul<eos>'

In [14]:
input_tokenized = tokenizer(full_converstion, return_tensors="pt", add_special_tokens=False).to(device)["input_ids"]
input_tokenized

tensor([[     2,    106,   1645,    108,  13033,    603,    573,   1963,   2040,
            577,   3918,   7472,  11716, 235336,    107,    108,    106,   2516,
            108,    651,   1963,   2040,    577,   3918,  16481,    603,   8720,
          11716, 151466,    954, 235276,    731, 122233,   3010,    578,  11239,
          95100,      1]])

In [15]:
input_ids = input_tokenized[:,:-1].to(device)
target_ids = input_tokenized[:, 1:].to(device)
print(f"input_ids: {input_ids}")
print(f"target_ids: {target_ids}")

input_ids: tensor([[     2,    106,   1645,    108,  13033,    603,    573,   1963,   2040,
            577,   3918,   7472,  11716, 235336,    107,    108,    106,   2516,
            108,    651,   1963,   2040,    577,   3918,  16481,    603,   8720,
          11716, 151466,    954, 235276,    731, 122233,   3010,    578,  11239,
          95100]])
target_ids: tensor([[   106,   1645,    108,  13033,    603,    573,   1963,   2040,    577,
           3918,   7472,  11716, 235336,    107,    108,    106,   2516,    108,
            651,   1963,   2040,    577,   3918,  16481,    603,   8720,  11716,
         151466,    954, 235276,    731, 122233,   3010,    578,  11239,  95100,
              1]])


In [16]:
import torch.nn as nn
def calculate_loss(logits, labels):
    loss_fct = nn.CrossEntropyLoss()
    loss = loss_fct(logits.view(-1, logits.shape[-1]), labels.view(-1))
    return loss

In [17]:
import torch
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16
    ).to(device)

config.json:   0%|          | 0.00/618 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [24]:
from torch.optim import AdamW
model.train()
optimizer = AdamW(model.parameters(), lr=1e-5, weight_decay=0.01)

for epochs in range(10):
  out = model(input_ids=input_ids)
  loss = calculate_loss(out.logits, target_ids).mean()
  loss.backward()
  optimizer.step()
  optimizer.zero_grad()
  print(loss.item())

4.03125
2.34375
2.0
1.703125
1.40625
1.1796875
1.125
1.078125
1.0390625
1.0078125


In [25]:
input_prompt = [
    {"role":"user", "content":"Which is the best place to learn GenAI?"}
]

input = tokenizer.apply_chat_template(
    conversation=input_prompt,
    return_tensors="pt",
    tokenize=True,
).to(device)

output = model.generate(input, max_new_tokens=20)
print(tokenizer.batch_decode(output, skip_special_tokens=True))

['user\nWhich is the best place to learn GenAI?\n оригіналу оригіналу is the best place to learn AI, but it requires significant time and effort.\nenumii']
